In [1]:
!sudo apt -y update
!sudo apt install -y imagemagick
!sudo cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

%pip install moviepy==2.0.0.dev2
%pip install imageio==2.25.1

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease    
Fetched 257 kB in 2s (126 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
All packages are up to date.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.5).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
/bin/bash: line 1: /etc/ImageMagick-6/policy.xml: Permission denied
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from moviepy.editor import TextClip

def create_caption(textJSON, framesize, font="Helvetica-Bold", fontsize=80, color='white', bgcolor='blue'):
    full_duration = textJSON['end'] - textJSON['start']

    word_clips = []
    xy_textclips_positions = []

    frame_width, frame_height = framesize
    x_buffer = frame_width * 1/10
    y_buffer = frame_height * 1/5

    # Calculate total width and height of the text
    total_width = 0
    total_height = 0
    current_line_width = 0
    max_line_width = frame_width - 2 * x_buffer
    line_height = 0
    
    for wordJSON in textJSON['words']:
        word_clip = TextClip(wordJSON['word'], font=font, fontsize=fontsize, color=color)
        word_width, word_height = word_clip.size
        space_width = TextClip(" ", font=font, fontsize=fontsize, color=color).w

        if current_line_width + word_width + space_width > max_line_width:
            total_height += line_height + 40
            total_width = max(total_width, current_line_width)
            current_line_width = word_width + space_width
            line_height = word_height
        else:
            current_line_width += word_width + space_width
            line_height = max(line_height, word_height)

    total_height += line_height
    total_width = max(total_width, current_line_width)

    # Calculate starting position to center the text
    start_x = (frame_width - total_width) / 2
    start_y = frame_height - total_height - y_buffer

    x_pos = start_x
    y_pos = start_y

    for wordJSON in textJSON['words']:
        duration = wordJSON['end'] - wordJSON['start']
        word_clip = TextClip(wordJSON['word'], font=font, fontsize=fontsize, color=color, stroke_color='black', stroke_width=2)
        word_clip_space = TextClip(" ", font=font, fontsize=fontsize, color=color)
        word_width, word_height = word_clip.size
        space_width = word_clip_space.w

        if x_pos + word_width + space_width > frame_width - x_buffer:
            x_pos = start_x
            y_pos += word_height + 40

        xy_textclips_positions.append({
            "x_pos": x_pos,
            "y_pos": y_pos,
            "width": word_width,
            "height": word_height,
            "word": wordJSON['word'],
            "start": wordJSON['start'],
            "end": wordJSON['end'],
            "duration": duration
        })

        word_clip = word_clip.set_position((x_pos, y_pos)).set_start(textJSON['start']).set_duration(full_duration)
        word_clip_space = word_clip_space.set_position((x_pos + word_width, y_pos)).set_start(textJSON['start']).set_duration(full_duration)

        word_clips.append(word_clip)
        word_clips.append(word_clip_space)

        x_pos += word_width + space_width

    for highlight_word in xy_textclips_positions:
        word_clip_highlight = TextClip(highlight_word['word'], font=font, fontsize=fontsize, color=bgcolor, stroke_color='black', stroke_width=2)
        word_clip_highlight = word_clip_highlight.set_position((highlight_word['x_pos'], highlight_word['y_pos'])).set_start(highlight_word['start']).set_duration(highlight_word['duration'])
        word_clips.append(word_clip_highlight)

    return word_clips

In [14]:
from moviepy.editor import TextClip, CompositeVideoClip, ImageClip, AudioFileClip, concatenate_videoclips
import os

def generate_karaoke_subtitle(transcript, backgrund_path, sound_path, base_path, output_file_name):
    # Define the path to the audio file
    video_path = os.path.join(base_path, f'{output_file_name}.mp4')

    if os.path.exists(video_path):
      return video_path
    
    # Load audio
    audio = AudioFileClip(sound_path)

    # Get the duration of the audio
    audio_duration = audio.duration

    image_clip = ImageClip(backgrund_path).set_duration(audio_duration)

    # Create a TextClip for each line of the transcript
    text_clips = []
    # Create a TextClip for each line of the transcript
    for line in transcript:
        text_clip = create_caption(line, image_clip.size)
        text_clips.extend(text_clip)

    # Create a CompositeVideoClip to combine the background and subtitle clips
    final_clip = CompositeVideoClip([image_clip] + text_clips).set_audio(audio)

    # Save the final clip as a video file with the audio included
    final_clip.write_videofile(video_path, fps=24, codec="libx264", audio_codec="aac")

    return video_path

In [17]:
import json
from PIL import Image
base_path = os.path.join(os.getcwd(), "sample")

with open(f"{base_path}/transcription.json", 'r') as f:
        transcript_data = json.load(f)


generate_karaoke_subtitle(transcript_data, f"{base_path}/background.jpg", f"{base_path}/music.mp3", base_path, "output")

{'id': 1, 'start': 7.12, 'end': 9.82, 'text': ' Hey, hey, hey.', 'words': [{'start': 7.12, 'end': 7.96, 'word': ' Hey,'}, {'start': 8.04, 'end': 8.38, 'word': ' hey,'}, {'start': 8.38, 'end': 9.82, 'word': ' hey.'}]}
{'id': 2, 'start': 9.94, 'end': 18.98, 'text': ' Bye, bye, bye, bye, bye, bye, bye, bye, bye, bye, bye, bye, bye, bye, bye, bye.', 'words': [{'start': 9.94, 'end': 10.36, 'word': ' Bye,'}, {'start': 10.5, 'end': 10.84, 'word': ' bye,'}, {'start': 10.84, 'end': 11.2, 'word': ' bye,'}, {'start': 11.52, 'end': 12.0, 'word': ' bye,'}, {'start': 12.0, 'end': 12.44, 'word': ' bye,'}, {'start': 12.62, 'end': 12.62, 'word': ' bye,'}, {'start': 12.62, 'end': 13.2, 'word': ' bye,'}, {'start': 13.76, 'end': 13.92, 'word': ' bye,'}, {'start': 17.16, 'end': 17.16, 'word': ' bye,'}, {'start': 17.8, 'end': 17.88, 'word': ' bye,'}, {'start': 17.88, 'end': 18.02, 'word': ' bye,'}, {'start': 18.02, 'end': 18.08, 'word': ' bye,'}, {'start': 18.08, 'end': 18.1, 'word': ' bye,'}, {'start': 18.

MoviePy - Done.
Moviepy - Writing video /home/jovyan/sample/output.mp4



Moviepy - Done !
Moviepy - video ready /home/jovyan/sample/output.mp4


'/home/jovyan/sample/output.mp4'